# Практические задания 20
* Лекция 38 - Базы данных. MongoDB  
**Онлайн-библиотека. Часть 2**
  
7. **Создание меню**  
Создайте простое пользовательское меню: пользователь может выбрать, какая информация его интересует, о книгах или о читателях. При выборе пункта меню о книгах запускается функция search_books_by_author(), при выборе другого пункта запускается функция search_books_by_client().  
Пример ввода (выделено красным) и вывода:
```
Меню:
1. Поиск информации о книгах
2. Поиск информации о клиентах
0. Выход
Выберите пункт меню (1, 2 или 0):  1
Введите имя автора (или его часть):  mar
```
Подходящие книги:
```
ID: 1008, Title: Life of Pi, Author: Yann Martel, Price: 0.95
ID: 4850, Title: The Book Thief, Author: Markus Zusak, Price: 0.32
```
```
Меню:
1. Поиск информации о книгах
2. Поиск информации о клиентах
0. Выход
Выберите пункт меню (1, 2 или 0):  2
Введите имя клиента (или его часть):  art
```
Клиенты и прочитанные ими книги:
не найдены.
```
Меню:
1. Поиск информации о книгах
2. Поиск информации о клиентах
0. Выход
Выберите пункт меню (1, 2 или 0):  leo
Некорректный выбор. Пожалуйста, выберите 1, 2 или 0.
```
```
Меню:
1. Поиск информации о книгах
2. Поиск информации о клиентах
0. Выход
Выберите пункт меню (1, 2 или 0):  0
Выход из программы.
```

In [ ]:
import pymysql

config = {
    'host': 'ich-edit.edu.itcareerhub.de',
    'user': 'ich1',
    'password': 'ich1_password_ilovedbs',
}

db_name = "library_test"


In [ ]:
def main_menu():
    while True:
        print("\nМеню:")
        print("1. Поиск информации о книгах")
        print("2. Поиск информации о клиентах")
        print("0. Выход")

        choice = input("Выберите пункт меню (1, 2 или 0): ")

        if choice == '1':
            search_books_by_author()
        elif choice == '2':
            search_books_by_client()
        elif choice == '0':
            print("Выход из программы.")
            break
        else:
            print("Некорректный выбор. Пожалуйста, выберите 1, 2 или 0.")

# Запуск главного меню
main_menu()



8. **Дополнительные запросы**  
Напишите дополнительные запросы:  
```
A. Поиск книг по части названия
B. Вычисление общего дохода библиотеки, сгененированного каждой книгой с упорядочиванием по убыванию дохода (пользователь должен иметь возможность указать количество строк вывода)
C. Вычисление суммарных затрат каждого пользователя с упорядочиванием по убыванию затрат (так же с указанием количества строк)

Пример ввода (выделено красным) и вывода: 
Введите часть названия книги:  love

Найденные книги:
ID: 2308, Title: Beloved, Author: Toni Morrison, Price: 0.62
______
Введите количество строк для вывода:  2

Общий доход, сгенерированный каждой книгой:
Author: Geoffrey Chaucer, Title: The Canterbury Tales, Total Income: 4.74
Author: Richard Osman, Title: The Thursday Murder Club, Total Income: 4.55
______
Введите количество строк для вывода:  3

Суммарные затраты каждого пользователя:
Mary Doe, Total Expenses: 6.09
David Wilson, Total Expenses: 4.89
James Jones, Total Expenses: 4.38
```

#### Решение A:

In [ ]:
def search_books_by_title():
    # Подключение к серверу MySQL
    with pymysql.connect(**config) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f"USE {db_name}")
            # Запрос части названия книги у пользователя
            title_part = input("Введите часть названия книги: ")

            # SQL-запрос для поиска книг по части названия
            query = """
                SELECT id, title, author, price
                FROM books
                WHERE title LIKE %s
            """
            cursor.execute(query, ('%' + title_part + '%',))

            # Получение результатов
            results = [res for res in cursor]

            # Вывод результатов
            if results:
                print("\nНайденные книги:")
                for row in results:
                    print(f"ID: {row[0]}, Title: {row[1]}, Author: {row[2]}, Price: {row[3]}")
            else:
                print("Книги не найдены.")

search_books_by_title()

#### Решение B:

In [ ]:
def calculate_total_income_by_book():
    # Подключение к серверу MySQL
    with pymysql.connect(**config) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f"USE {db_name}")
            # Запрос количества строк для вывода у пользователя
            limit = int(input("Введите количество строк для вывода: "))

            # SQL-запрос для вычисления общего дохода, сгенерированного каждой книгой
            query = """
                SELECT b.author, b.title, SUM(b.price) AS total_income
                FROM reading r
                JOIN books b ON r.bookid = b.id
                GROUP BY b.author, b.title
                ORDER BY total_income DESC
                LIMIT %s
            """
            cursor.execute(query, (limit,))

            # Вывод результатов
            print("\nОбщий доход, сгенерированный каждой книгой:")
            for row in cursor:
                print(f"Author: {row[0]}, Title: {row[1]}, Total Income: {row[2]}")

calculate_total_income_by_book()


#### Решение C:

In [ ]:
def calculate_total_expenses_by_user():
    # Подключение к серверу MySQL
    with pymysql.connect(**config) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f"USE {db_name}")
            # Запрос количества строк для вывода у пользователя
            limit = int(input("Введите количество строк для вывода: "))

            # SQL-запрос для вычисления суммарных затрат каждого пользователя
            query = """
                SELECT u.firstname, u.lastname, SUM(b.price) AS total_expenses
                FROM reading r
                JOIN users u ON r.userid = u.id
                JOIN books b ON r.bookid = b.id
                GROUP BY u.firstname, u.lastname
                ORDER BY total_expenses DESC
                LIMIT %s
            """
            cursor.execute(query, (limit,))

            # Вывод результатов
            print("\nСуммарные затраты каждого пользователя:")
            for row in cursor:
                print(f"{row[0]} {row[1]}, Total Expenses: {row[2]}")

calculate_total_expenses_by_user()


9. **Двухуровневое меню**  
Перепишите функцию main_menu(). Сделайте меню двухуровневым: после пункта о поиске информации о книгах должно появляться меню выбора поиска (по автору или названию) или вывода информации о доходах. При выборе пункта меню о поиске информации о клиентах должен быть предоставлен выбор поиска информации о взятых книгах конкретного клиента или вывод информации о затратах.
Пример ввода (выделено красным) и вывода:   
```
Главное меню:
1. Поиск информации о книгах
2. Поиск информации о клиентах
0. Выход
Выберите пункт меню (1, 2 или 3):  1

Меню информации о книгах:
1. Поиск книг по автору
2. Поиск книг по части названия
3. Общий доход, сгенерированный каждой книгой
0. Назад
Выберите пункт подменю (1, 2, 3 или 4):  2
Введите часть названия книги:  rose

Найденные книги:
ID: 1117, Title: The Rose Code, Author: Kate Quinn, Price: 0.88

Меню информации о книгах:
1. Поиск книг по автору
2. Поиск книг по части названия
3. Общий доход, сгенерированный каждой книгой
0. Назад
Выберите пункт подменю (1, 2, 3 или 4):  0

Главное меню:
1. Поиск информации о книгах
2. Поиск информации о клиентах
0. Выход
Выберите пункт меню (1, 2 или 3):  0
Выход из программы.
```

In [ ]:
def main_menu():
    while True:
        print("\nГлавное меню:")
        print("1. Поиск информации о книгах")
        print("2. Поиск информации о клиентах")
        print("0. Выход")

        choice = input("Выберите пункт меню (1, 2 или 0): ")

        if choice == '1':
            while True:
                print("\nМеню информации о книгах:")
                print("1. Поиск книг по автору")
                print("2. Поиск книг по части названия")
                print("3. Общий доход, сгенерированный каждой книгой")
                print("0. Назад")

                sub_choice = input("Выберите пункт подменю (1, 2, 3 или 0): ")

                if sub_choice == '1':
                    search_books_by_author()
                elif sub_choice == '2':
                    search_books_by_title()
                elif sub_choice == '3':
                    calculate_total_income_by_book()
                elif sub_choice == '0':
                    break
                else:
                    print("Некорректный выбор. Пожалуйста, выберите 1, 2, 3 или 0.")

        elif choice == '2':
            while True:
                print("\nМеню информации о клиентах:")
                print("1. Поиск книг клиента")
                print("2. Суммарные затраты каждого клиента")
                print("0. Назад")

                sub_choice = input("Выберите пункт подменю (1, 2 или 0): ")

                if sub_choice == '1':
                    search_books_by_client()
                elif sub_choice == '2':
                    calculate_total_expenses_by_user()
                elif sub_choice == '0':
                    break
                else:
                    print("Некорректный выбор. Пожалуйста, выберите 1, 2 или 0.")

        elif choice == '0':
            print("Выход из программы.")
            break
        else:
            print("Некорректный выбор. Пожалуйста, выберите 1, 2 или 0.")

main_menu()


10. **Фиксация поисков в MongoDB**  
Добавьте сохранение всех поисковых запросов по книгам и пользователям в MongoDB для последующего анализа.  
Используйте подключение:   
```
from pymongo import MongoClient

client = MongoClient(
    "mongodb://ich_editor:verystrongpassword"
    "@mongo.itcareerhub.de/?readPreference=primary"
    "&ssl=false&authMechanism=DEFAULT&authSource=ich_edit"
)

Используйте базу ich_edit, коллекцию library_<номер группы>_logs_searches;
При каждом поиске сохраняйте вид и текст запроса — то, что ввёл пользователь;
Пример записи в MongoDB: 
{ "query_type": "function_name", "query_str": "user_str" }
```

#### Решение (доработка search_books_by_title): 

In [ ]:
from pymongo import MongoClient

# Настройка подключения к MongoDB
mongo_client = MongoClient(
    "mongodb://ich_editor:verystrongpassword"
    "@mongo.itcareerhub.de/?readPreference=primary"
    "&ssl=false&authMechanism=DEFAULT&authSource=ich_edit"
)
searches_collection = mongo_client["ich_edit"]["library_110425_logs_searches"]

def log_search_query(query_type, query_str):
    searches_collection.insert_one({"query_type": query_type, "query_str": query_str})


def search_books_by_title():
    # фиксируем запрос в MongoDB
    log_search_query("search_books_by_title", title_part)


In [ ]:
title_part = 'love'

search_books_by_title()

11. **Самые популярные поисковые запросы**  
Добавьте в меню возможность вывести самые частые поисковые запросы, которые пользователи вводили при поиске книг.  
* Используйте данные из MongoDB;
* Сгруппируйте запросы по полю query_type;
* Выведите запросы в порядке убывания количества;
* Покажите только топ-5 самых популярных запросов;
* Выводите также количество повторений для каждого.  
Пример вывода:   
```
Most frequent search queries:
1. search_books_by_title — 7 times
2. search_books_by_client — 5 times
3. calculate_total_expenses_by_user — 3 times
4. search_books_by_author — 2 times
5. calculate_total_income_by_book — 2 times
```

#### Решение (метод для анализа):


In [ ]:
from collections import Counter

def show_popular_queries():
    queries = [doc["query_type"] for doc in searches_collection.find() if doc.get("query_type")]
    counter = Counter(queries)
    top = counter.most_common(5)

    if not top:
        print("No search data found.")
        return

    print("Most frequent search queries:")
    for i, (query, count) in enumerate(top):
        print(f"{i+1}. {query} — {count} times")

show_popular_queries()



#### Решение (добавление в меню – существенные изменения выделены красным):

In [ ]:
def main_menu():
    while True:
        print("\nГлавное меню:")
        print("1. Поиск информации о книгах")
        print("2. Поиск информации о клиентах")
        print("3. Вывод топ-5 популярных запросов")
        print("0. Выход")

        choice = input("Выберите пункт меню (1, 2, 3 или 0): ")

        if choice == '1':
            while True:
                print("\nМеню информации о книгах:")
                print("1. Поиск книг по автору")
                print("2. Поиск книг по части названия")
                print("3. Общий доход, сгенерированный каждой книгой")
                print("0. Назад")

                sub_choice = input("Выберите пункт подменю (1, 2, 3 или 0): ")

                if sub_choice == '1':
                    search_books_by_author()
                elif sub_choice == '2':
                    search_books_by_title()
                elif sub_choice == '3':
                    calculate_total_income_by_book()
                elif sub_choice == '0':
                    break
                else:
                    print("Некорректный выбор. Пожалуйста, выберите 1, 2, 3 или 0.")

        elif choice == '2':
            while True:
                print("\nМеню информации о клиентах:")
                print("1. Поиск книг клиента")
                print("2. Суммарные затраты каждого клиента")
                print("0. Назад")

                sub_choice = input("Выберите пункт подменю (1, 2 или 0): ")

                if sub_choice == '1':
                    search_books_by_client()
                elif sub_choice == '2':
                    calculate_total_expenses_by_user()
                elif sub_choice == '0':
                    break
                else:
                    print("Некорректный выбор. Пожалуйста, выберите 1, 2 или 0.")

        elif choice == '3':
            show_popular_queries()

        elif choice == '0':
            print("Выход из программы.")
            break
        else:
            print("Некорректный выбор. Пожалуйста, выберите 1, 2, 3 или 0.")


